## Making the trianable ansatz

In [1]:
import numpy as np

In [2]:
def nestedKronecker(args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

basis = {0: [1,0], 1: [0,1], '0': [1,0], '1': [0,1]}

basisVector = lambda binstr : nestedKronecker([basis[x] for x in binstr])

# common states
zero, one = basis['0'], basis['1']
tplus = basisVector('11')
tminus = basisVector('00')
tzero = (1/np.sqrt(2))*(basisVector('01') + basisVector('10'))
singlet = np.sqrt(1/2)*(basisVector('01') - basisVector('10'))


# ------------------------ FOR STATE 1 ------------------------

state1 = np.kron(np.kron(singlet, singlet), singlet)

# ------------------------ FOR STATE 2 ------------------------

largePyramid = np.sqrt(1/3)*(np.kron(tplus,tminus)+np.kron(tminus,tplus)-np.kron(tzero,tzero))
state2 = np.kron(singlet,largePyramid)

# ------------------------ FOR STATE 3 ------------------------

state3 = np.kron(largePyramid,singlet)

# ------------------------ FOR STATE 4 ------------------------

# for psi0 and psi1 we are combining j1=1 and j2=1/2 (this is combinind the first peak and trough)
# J = 1/2, M = -1/2
psi0 = np.sqrt(1/3)*np.kron(tzero, zero) - np.sqrt(2/3)*np.kron(tminus, one)
# J = 1/2, M = +1/2
psi1 = np.sqrt(2/3)*np.kron(tplus, zero) - np.sqrt(1/3)*np.kron(tzero, one)


# for phiminus, phizero, phiplus, we are are combining j1=1/2 and j2=1/2
# J = 1, M = -1
phiminus = np.kron(psi0,zero)
# J = 1, M = 0
phizero = np.sqrt(1/2)*(np.kron(psi1,zero) + np.kron(psi0,one))
# J = 1, M = +1
phiplus = np.kron(psi1,one)

# J=0,M=0 and j1=1,j2=1
state4 = np.sqrt(1/3)*(np.kron(phiplus, tminus) - np.kron(phizero, tzero) + np.kron(phiminus, tplus))

# ------------------------ FOR STATE 5 ------------------------

eta_minus3 = np.kron(tminus, basis['0'])
eta_minus1 = np.sqrt(2/3)*np.kron(tzero,zero) + np.sqrt(1/3)*np.kron(tminus,one)
eta_plus1 = np.sqrt(1/3)*np.kron(tplus,zero) + np.sqrt(2/3)*np.kron(tzero, one)
eta_plus3 = np.kron(tplus,one)

gamma_minus = np.sqrt(1/4)*np.kron(eta_minus1, zero) - np.sqrt(3/4)*np.kron(eta_minus3, one)
gamma_zero = np.sqrt(1/2)*np.kron(eta_plus1, zero) - np.sqrt(1/2)*np.kron(eta_minus1,one)
gamma_plus = np.sqrt(3/4)*np.kron(eta_plus3, zero) - np.sqrt(1/4)*np.kron(eta_plus1, one)

state5 = np.sqrt(1/3)*(np.kron(gamma_plus,tminus) - np.kron(gamma_zero, tzero) - np.kron(gamma_minus, tplus))

inputStates = np.array([state1, state2, state3, state4])
expectedStates = np.array([state1, state2, state4, state3])

# Trying ChatPGT

In [67]:
import numpy as np
from scipy.optimize import minimize
import pennylane as qml

I = np.eye(2)
n_qubits = 6
size_of_vec = 2**n_qubits
num_layers = 5
inputStates = np.array([state1, state2, state3, state4])
expectedStates = np.array([state1, state2, state4, state3])
prcnt_drop = 17


Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

def nestedKron(*args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

def get_random_weights(num_layers,prcnt_drop):
    percent_to_zero = prcnt_drop
    random_array = np.random.rand(5, num_layers)
    random_array = 2 * np.pi * random_array - np.pi
    mask = np.random.choice([0, 1], size=(5, num_layers), p=[percent_to_zero / 100, (100 - percent_to_zero) / 100])
    return random_array * mask

def U_ex(p):
    from scipy.linalg import expm
    X = [[0,1],[1,0]]
    Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
    Z = [[1,0],[0,-1]]

    H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))
    # print(f'H_ex.type = {type(H_ex)}')
    U_exchange = expm(-1j*p*H_ex) # p is now -pi to pi
    return np.array(U_exchange)

def single_layer_U(layer_weights):
    """Trainable circuit block."""
    firstPart = nestedKron(U_ex(layer_weights[0]), U_ex(layer_weights[1]), U_ex(layer_weights[2]))
    secondPart = nestedKron(I, U_ex(layer_weights[3]), U_ex(layer_weights[4]), I)
    return np.matmul(secondPart, firstPart)

def get_matrix(weights):
    totalMatrix = np.eye(size_of_vec)
    for layer_weights in weights:
        mat = single_layer_U(layer_weights)
        totalMatrix = np.matmul(totalMatrix, mat)
    return totalMatrix

def square_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        # c = np.dot(np.conjugate(expectedStates[i]), predictedStates[i])
        # c_2 = self.amp_sqrd(c)
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += (1 - fidelity) ** 2
    loss /= len(expectedStates)
    return 0.5*loss


def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))

# Define the correct operations you want the matrix to perform on basis vectors
def target_operations(non_zero_parameters, inputStates, A):
    param = np.zeros_like(A.flatten())
    param[A.flatten() != 0] = non_zero_parameters
    parameters = np.reshape(param, (num_layers, 5))
    matrix = get_matrix(parameters)
    # Perform matrix multiplication with basis vectors
    results = []
    for i in range(len(inputStates)):
        results.append(np.matmul(matrix, inputStates[i]))

    # Define the target operations you want (modify this based on your specific task)
    target_result = np.array(expectedStates)

    # Calculate the loss as the difference between the obtained result and the target result
    loss = f_cnot_loss(target_result,results)#square_loss(target_result, results)
    return loss

# Example: Set the number of basis vectors and their dimensionality
num_vectors = 4
vector_dimension = size_of_vec

# Generate random basis vectors and target result
basis_vectors = np.array(inputStates)
target_result = np.array(expectedStates)

# Flatten the matrix parameters for optimization
initial_parameters = get_random_weights(num_layers,prcnt_drop)
non_zero_parameters = initial_parameters.flatten()[initial_parameters.flatten() != 0]
print(initial_parameters)
# Use scipy's minimize function to optimize the parameters
result = minimize(target_operations,non_zero_parameters, args=(basis_vectors,initial_parameters,), method='L-BFGS-B')
#print(result.x)
# Reshape the optimized parameters back into the matrix form
param = np.zeros_like(initial_parameters.flatten())
param[initial_parameters.flatten() != 0] = result.x
optimized_results = np.reshape(param, (num_layers, 5))
print(optimized_results)
optimized_matrix = get_matrix(optimized_results)

print("Optimized Matrix:")
#for i in optimized_matrix:
#    print(i)
print(optimized_matrix)

predStates = [np.matmul(optimized_matrix, mat) for mat in inputStates]
print(f"fcnot loss = {f_cnot_loss(expectedStates, predStates)}")
#print(f"square loss = {square_loss(expectedStates, predStates)}")


[[ 1.15115758  0.64894976 -2.7497892  -1.56514938 -0.3248914 ]
 [ 2.74246052  0.         -1.04444073 -0.43276333 -1.79710542]
 [-1.4378175   0.39262192  2.30767816  0.          0.36155878]
 [-3.00532631  1.71912356 -0.         -1.490466    1.19882376]
 [ 2.36881599 -0.         -2.73885275  0.         -0.01882193]]
[[ 2.02476993e+00 -1.39165575e+00 -2.24084724e+00  8.23960319e-09
  -9.25868544e-01]
 [ 3.23124800e+00  0.00000000e+00 -1.37774309e+00  6.40500513e-09
  -2.86396239e+00]
 [-9.49025806e-01 -2.37852617e+00  2.28364398e+00  0.00000000e+00
  -1.27453800e+00]
 [-3.00532672e+00  3.69304453e+00  0.00000000e+00  2.34217013e-09
   1.73770713e+00]
 [ 2.36881642e+00  0.00000000e+00 -2.73885174e+00  0.00000000e+00
  -1.64839381e+00]]
Optimized Matrix:
[[ 0.20544562+0.97866853j  0.        +0.j          0.        +0.j
  ...  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.3328996 -0.1440384j  -0.17823361+0.41220946j
  ...  0.        +0.j      

In [5]:
from QNN import Ansatz
ansatz = Ansatz(num_layers=num_layers)
ansatz.weights = result.x.reshape((num_layers, 5))
predStates = ansatz.get_predictions(ansatz.weights, inputStates)
loss = ansatz.square_loss(expectedStates, predStates)

print(f'loss from the pennylane qml model is = {loss}')


loss from the pennylane qml model is = 0.27960364502846224
